# 주제 : LSTM을 활용해 주가 예측 모델 구현하기

이번 튜토리얼 에서는 다음과 같은 **프로세스 파이프라인**으로 주가 예측을 진행합니다.

- FinanceDataReader를 활용하여 주가 데이터 받아오기
- TensorFlow Dataset 클래스를 활용하여 주가 데이터 구축
- LSTM 을 활용한 주가 예측 모델 구축

## Step 1. 데이터 불러오기 및 EDA

### 문제 01. 필요한 모듈 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

%matplotlib inline
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothic'

### 문제 02. FinanceDataReader import

**FinanceDataReader**는 주가 데이터를 편리하게 가져올 수 있는 파이썬 패키지입니다.

- [GitHub Repo](https://github.com/FinanceData/FinanceDataReader)

**FinanceDataReader**가 아직 설치 되지 않으신 분들은 아래의 주석을 해제한 후 명령어로 설치해 주시기 바랍니다.

In [2]:
# !pip install finance-datareader

You should consider upgrading via the 'c:\users\상혁\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import FinanceDataReader as fdr

### 문제 03. 삼성전자 데이터 불러오기

In [5]:
# fdr 라이브러리를 활용해 삼성전자 주가 데이터를 불러오세요.
# 대상 : 삼성전자(005930) 전체 (1996-11-05 ~ 현재)
samsung = fdr.DataReader('005930')

In [6]:
samsung.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
1997-06-27,1256,1265,1152,1224,99340,NaN
1997-06-28,1227,1250,1224,1251,36660,0.022059
1997-06-30,1259,1290,1254,1260,100240,0.007194
1997-07-01,1259,1286,1259,1269,167900,0.007143
1997-07-02,1277,1313,1277,1296,169940,0.021277


매우 편리하게 삼성전자 주가 데이터를 `DataFrame`형식으로 받아옵니다.

기본 **오름차순 정렬**이 된 데이터임을 알 수 있습니다.

### 컬럼 설명

- `Open`:   시가
- `High`:   고가
- `Low`:    저가
- `Close`:  종가
- `Volume`: 거래량
- `Change`: 대비

### 문제 04. 삼성전자 데이터 EDA

In [7]:
samsung.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-06-28,81700,82000,81600,81900,11578529,0.003676
2021-06-29,81900,82100,80800,81000,15744317,-0.010989
2021-06-30,81100,81400,80700,80700,13288643,-0.003704
2021-07-01,80500,80600,80000,80100,13382882,-0.007435
2021-07-02,80000,80400,79900,80000,8181692,-0.001248


In [8]:
samsung.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6000 entries, 1997-06-27 to 2021-07-02
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    6000 non-null   int64  
 1   High    6000 non-null   int64  
 2   Low     6000 non-null   int64  
 3   Close   6000 non-null   int64  
 4   Volume  6000 non-null   int64  
 5   Change  5999 non-null   float64
dtypes: float64(1), int64(5)
memory usage: 328.1 KB


In [9]:
samsung.describe()

,Open,High,Low,Close,Volume,Change
count,6000.000000,6000.000000,6000.000000,6000.000000,6.000000e+03,5999.000000
mean,20730.173167,20954.542500,20505.031667,20755.953833,2.544814e+06,0.001027
std,18213.883595,18385.055374,18046.507131,18218.540457,6.283996e+06,0.025757
min,0.000000,0.000000,0.000000,627.000000,0.000000e+00,-0.137566
25%,7400.000000,7500.000000,7277.250000,7400.000000,2.749732e+05,-0.012231
50%,13730.000000,13900.000000,13570.000000,13760.000000,4.492690e+05,0.000000
75%,28000.000000,28220.000000,27670.000000,28000.000000,8.159305e+05,0.012965
max,90300.000000,96800.000000,89500.000000,91000.000000,9.030618e+07,0.150000


**미국 주식 데이터**도 가져올 수 있습니다.

### 문제 05. 애플 데이터 불러오기

In [11]:
# fdr 라이브러리를 활용해 Apple(AAPL) 데이터를 불러오세요.
apple = fdr.DataReader("AAPL")

### 문제 06. 애플 데이터 EDA

In [12]:
apple.head()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
1980-12-12,0.13,0.13,0.13,0.13,469030000.0,-0.9988
1980-12-15,0.12,0.12,0.12,0.12,175880000.0,-0.0769
1980-12-16,0.11,0.11,0.11,0.11,105730000.0,-0.0833
1980-12-17,0.12,0.12,0.12,0.12,86440000.0,0.0909
1980-12-18,0.12,0.12,0.12,0.12,73450000.0,0.0000


In [13]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10226 entries, 1980-12-12 to 2021-07-02
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   10226 non-null  float64
 1   Open    10226 non-null  float64
 2   High    10226 non-null  float64
 3   Low     10226 non-null  float64
 4   Volume  10226 non-null  float64
 5   Change  10226 non-null  float64
dtypes: float64(6)
memory usage: 559.2 KB


In [14]:
apple.describe()

,Close,Open,High,Low,Volume,Change
count,10226.000000,10226.000000,10226.000000,10226.000000,1.022600e+04,10226.000000
mean,11.399781,11.397103,11.518780,11.272060,3.366108e+08,0.001044
std,23.463528,23.459847,23.721381,23.185748,3.412730e+08,0.034704
min,0.050000,0.050000,0.050000,0.050000,1.390000e+06,-0.998800
25%,0.280000,0.280000,0.280000,0.270000,1.282375e+08,-0.008200
50%,0.450000,0.450000,0.460000,0.440000,2.241000e+08,0.000000
75%,12.380000,12.370000,12.490000,12.310000,4.188400e+08,0.012275
max,143.160000,143.600000,145.090000,141.370000,7.430000e+09,0.277800


### 문제 07. 특정 시점(2017년) 이후 데이터 불러오기

In [15]:
# Apple(AAPL), 2017년
apple = fdr.DataReader('AAPL', '2017')

In [16]:
apple.head()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2017-01-03,29.04,28.95,29.08,28.69,115130000.0,0.0031
2017-01-04,29.00,28.96,29.13,28.94,84470000.0,-0.0014
2017-01-05,29.15,28.98,29.22,28.95,88770000.0,0.0052
2017-01-06,29.48,29.20,29.54,29.12,127010000.0,0.0113
2017-01-09,29.75,29.49,29.86,29.48,134250000.0,0.0092


### 문제 08. 날짜를 지정하여 특정 범위(40년간) 데이터 불러오기

In [17]:
# Ford(F), 1980-01-01 ~ 2019-12-30 (40년 데이터)
ford = fdr.DataReader('F', '1980-01-01', '2019-12-30')

In [18]:
ford.head()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
1980-03-18,1.83,1.83,1.85,1.82,3770000.0,-0.0108
1980-03-19,1.85,1.85,1.86,1.83,1560000.0,0.0109
1980-03-20,1.86,1.86,1.88,1.85,1450000.0,0.0054
1980-03-21,1.78,1.78,1.85,1.76,5020000.0,-0.0430
1980-03-24,1.71,1.71,1.75,1.66,3330000.0,-0.0393


In [19]:
ford.tail()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2019-12-23,9.44,9.50,9.57,9.40,54800000.0,-0.0042
2019-12-24,9.47,9.44,9.49,9.43,11880000.0,0.0032
2019-12-26,9.45,9.47,9.49,9.43,28980000.0,-0.0021
2019-12-27,9.36,9.45,9.46,9.35,28270000.0,-0.0095
2019-12-30,9.25,9.34,9.35,9.23,36090000.0,-0.0118


### 문제 09. '금'과 '달러' 데이터도 가져올 수 있습니다. 두 데이터를 불러와 head를 출력해보세요.

*[GitHub 페이지 링크](https://github.com/FinanceData/FinanceDataReader) 참고

In [24]:
# 금 선물 가격
gold = fdr.DataReader('ZG')

In [25]:
gold.head()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
1980-01-02,575.5,562.5,577.0,558.0,7260.0,0.0785
1980-01-03,625.0,627.0,640.0,603.0,20750.0,0.0860
1980-01-04,603.6,609.5,621.0,573.0,2660.0,-0.0342
1980-01-07,627.0,629.0,635.0,611.0,3000.0,0.0388
1980-01-08,602.5,611.5,619.0,600.0,750.0,-0.0391


In [26]:
gold.tail()

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2021-06-25,1777.8,1775.7,1791.0,1773.6,161200.0,0.0006
2021-06-28,1780.7,1782.0,1786.1,1770.4,160790.0,0.0016
2021-06-29,1763.6,1778.8,1779.2,1750.1,239910.0,-0.0096
2021-06-30,1771.6,1761.9,1774.7,1753.2,177890.0,0.0045
2021-07-01,1776.8,1770.8,1783.4,1765.9,171440.0,0.0029


In [21]:
# 달러 가격
dollar = fdr.DataReader('USD/KRW')

In [22]:
dollar.head()

,Close,Open,High,Low,Change
Date,,,,,
1981-04-14,676.70,676.70,676.70,676.70,0.0019
1981-04-15,676.70,676.70,676.70,676.70,0.0000
1981-04-16,676.70,676.70,676.70,676.70,0.0000
1981-04-17,676.75,676.75,676.75,676.75,0.0001
1981-04-20,676.70,676.70,676.70,676.70,-0.0001


In [23]:
dollar.tail()

,Close,Open,High,Low,Change
Date,,,,,
2021-06-28,1130.04,1127.34,1132.68,1126.58,0.0026
2021-06-29,1131.62,1130.21,1134.77,1127.65,0.0014
2021-06-30,1130.48,1131.80,1132.89,1125.59,-0.0010
2021-07-01,1134.33,1130.63,1135.98,1128.28,0.0034
2021-07-02,1133.70,1134.19,1137.91,1132.26,-0.0006


### 문제 10. 삼성전자 데이터를 STOCK_CODE에 저장하고 인덱스를 확인해보세요

In [27]:
# 문제 03과 코드는 동일합니다.
# 삼성전자 주식코드: 005930
STOCK_CODE = '005930'

In [28]:
# fdr 라이브러리를 활용해 삼성전자 데이터를 불러오세요 
stock = fdr.DataReader(STOCK_CODE)
stock.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
1997-06-27,1256,1265,1152,1224,99340,NaN
1997-06-28,1227,1250,1224,1251,36660,0.022059
1997-06-30,1259,1290,1254,1260,100240,0.007194
1997-07-01,1259,1286,1259,1269,167900,0.007143
1997-07-02,1277,1313,1277,1296,169940,0.021277


In [29]:
stock.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-06-28,81700,82000,81600,81900,11578529,0.003676
2021-06-29,81900,82100,80800,81000,15744317,-0.010989
2021-06-30,81100,81400,80700,80700,13288643,-0.003704
2021-07-01,80500,80600,80000,80100,13382882,-0.007435
2021-07-02,80000,80400,79900,80000,8181692,-0.001248


In [32]:
# 인덱스를 확인해보세요.
stock.index

DatetimeIndex(['1997-06-27', '1997-06-28', '1997-06-30', '1997-07-01',
               '1997-07-02', '1997-07-03', '1997-07-04', '1997-07-05',
               '1997-07-07', '1997-07-08',
               ...
               '2021-06-21', '2021-06-22', '2021-06-23', '2021-06-24',
               '2021-06-25', '2021-06-28', '2021-06-29', '2021-06-30',
               '2021-07-01', '2021-07-02'],
              dtype='datetime64[ns]', name='Date', length=6000, freq=None)

## Step 2. 시계열 데이터 시각화


### 문제 11. 인덱스가 `DatetimeIndex`로 정의되어 있다면, 연도, 월, 일을 쪼갤 수 있습니다. 데이터를 분리해주세요.

In [ ]:
stock['Year'] = 
stock['Month'] = 
stock['Day'] = 

### 문제 12. 연도별, 월별 피봇테이블을 각각 구현해보세요.

### 문제 13. matplotlib을 활용해 시간에 따른 주식 가격 시계열 그래프를 그려보세요.

In [ ]:
plt.figure(figsize=(16, 9))
# 코드를 이어서 작성해주세요.


### 문제 14. subplots를 활용해 그래프 4개(2,2)를 동시에 그려보세요.

In [ ]:
# 1990~2000, 2000~2010, 2010~2015, 2015~2020
time_steps = 

fig, axes = 

for i in range(4):
    ax = #코드를 작성해주세요
    df = #코드를 작성해주세요
plt.tight_layout()
plt.show()

## Step 3. 시계열 데이터의 전처리

### 문제 15. MinMaxScaler를 활요해 데이터 전처리를 수행합니다.

주가 데이터에 대하여 딥러닝 모델이 더 잘 학습할 수 있도록 **정규화(Normalization)**를 해주도록 하겠습니다.

**표준화 (Standardization)**와 **정규화(Normalization)**에 대한 내용은 아래 링크에서 더 자세히 다루니, 참고해 보시기 바랍니다.

- [데이터 전처리에 관하여](https://teddylee777.github.io/scikit-learn/scikit-learn-preprocessing)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = #MinMaxScaler 할당
# 스케일을 적용할 column을 정의합니다.
scale_cols = 
# 스케일 후 columns
scaled = 

스케일이 완료된 column으로 새로운 데이터프레임을 생성합니다.

**시간 순으로 정렬**되어 있으며, datetime index는 제외했습니다.

*6,000개의 row, 5개 column*으로 이루어진 데이터셋이 DataFrame으로 정리되었습니다.

In [ ]:
# 아래 코드는 그대로 실행해주세요.
df = pd.DataFrame(scaled, columns=scale_cols)

### 문제 16. 모델 학습을 위해 train 데이터와 test 데이터로 분할해주세요.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Close 컬럼이 예측할 데이터입니다.
# test_size는 0.2, random_state는 0으로 지정해주세요.
# shuffle을 수행하지 않습니다.
x_train, x_test, y_train, y_test = 

### 문제 17. train 데이터와 test 데이터의 shape을 각각 살펴봅니다.

### 문제 18. TensroFlow Dataset을 활용해 시퀀스 데이터셋을 함수로 구현해주세요

In [ ]:
import tensorflow as tf

In [ ]:
def windowed_dataset():
    series = 
    ds = 
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(#코드 작성)
    return ds.batch(batch_size).prefetch(1)

## Step 4. 모델 구현하기

### 문제 19. Hyperparameter를 정의하고 데이터에 적용해주세요.

In [ ]:
# 아래 코드는 그대로 실행해주세요.
WINDOW_SIZE=20
BATCH_SIZE=32

In [ ]:
# trian_data는 학습용 데이터셋, test_data는 검증용 데이터셋 입니다.
# WINDOW_SIZE와 BATCH_SIZE를 각 데이터셋에 적용해주세요.
train_data = windowed_dataset(#코드 작성
test_data = windowed_dataset(#코드 작성

In [ ]:
# 아래의 코드로 데이터셋의 구성을 확인해 볼 수 있습니다. 그대로 실행해보고, 데이터의 shape을 살펴보세요.
# X: (batch_size, window_size, feature)
# Y: (batch_size, feature)
for data in train_data.take(1):
    print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
    print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')

데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 20, 1)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 1)


### 문제 20. Sequential 모델을 구현합니다. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential([
    # 1차원 feature map 생성합니다. filters는 32로, kernel_size는 5로 지정해주세요.
    

    # LSTM과 Dense 레이어를 사용해주세요. 필요한 경우 활성함수는 relu로 지정합니다.
    
])

### 문제 21. 모델을 Compile합니다. loss는 Huber 함수를 사용하고 optimizer는 Adam을 사용해주세요.

In [ ]:
# Sequence 학습에 비교적 좋은 퍼포먼스를 내는 Huber()를 사용합니다.
loss = 

model.compile()

In [ ]:
# earlystopping은 10번 epoch통안 val_loss 개선이 없다면 학습을 멈춥니다.
earlystopping = 
# val_loss 기준 체크포인터도 생성합니다.
filename = 
checkpoint = ModelCheckpoint()

In [ ]:
# callbacks로 앞에서 구현한 earlystopping과 checkpoint를 지정해주세요.
history =

## Step 5. 모델을 활용한 예측 및 결과 시각화

### 문제 24. 저장한 ModelCheckpoint 를 불러옵니다.

### 문제 25. `test_data`를 활용하여 예측을 수행합니다.

In [ ]:
pred =

### 문제 26. matplotlib을 활용해 예측 데이터를 시각화해주세요.

아래 시각화 코드중 y_test 데이터에 **[20:]**으로 슬라이싱을 한 이유는

예측 데이터에서 20일치의 데이터로 21일치를 예측해야하기 때문에 test_data로 예측 시 *앞의 20일은 예측하지 않습니다.*

따라서, 20번 째 index와 비교하면 더욱 정확합니다.

In [ ]:
# 20일치의 데이터로 21일치를 예측하므로 test_data 사용시 이전 20일은 예측하지 않습니다.
# 따라서 y_test 데이터에 [20:]로 슬라이싱해주세요.
plt.figure(figsize=(12, 9))


plt.legend()
plt.show()